Importing dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Data collection and pre-processing

In [2]:
# Loading data from csv file to pandas DataFrame
raw_mail_data = pd.read_csv('mail_data.csv')

In [3]:
print(raw_mail_data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [4]:
# Replace null values with a null string
mail_data = raw_mail_data.where((pd.notnull(raw_mail_data)),'')

In [5]:
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
# Checking number of rows and columns in DataFrame
mail_data.shape

(5572, 2)

Label Encoding

In [7]:
# Label spam mail as 0, ham mail as 1
mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

In [8]:
# Separating the data as texts and labels
X = mail_data['Message']
Y = mail_data['Category']

In [9]:
print(X)
print(Y)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object
0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


Splitting data into training and test data

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

In [11]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5572,)
(4457,)
(1115,)


Feature extraction

In [12]:
# Convert text data to feature vectors using TF-IDF (Term Frequency-Inverse Document Frequency)
# This transformation helps in converting the raw text into numerical values that can be used by the model
feature_extraction = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)

# Apply transformation to both training and test data
X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

# Convert the labels (Y) from object to integers to be used by the model
Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [13]:
print(X_train)

5281               And how you will do that, princess? :)
1300    Great to hear you are settling well. So what's...
5128    Wot about on wed nite I am 3 then but only til 9!
1808      Do have a nice day today. I love you so dearly.
856     Talk sexy!! Make new friends or fall in love i...
                              ...                        
1180                  To day class is there are no class.
3441                       What time you thinkin of goin?
1344    Crazy ar he's married. Ü like gd looking guys ...
4623                They finally came to fix the ceiling.
1289                             Happy new year to u too!
Name: Message, Length: 4457, dtype: object


In [14]:
print(X_train_features)

  (0, 5218)	1.0
  (1, 3117)	0.2702256012963159
  (1, 3258)	0.33769704827191216
  (1, 5831)	0.47563189358898533
  (1, 3213)	0.47563189358898533
  (1, 7266)	0.39349136612703317
  (1, 4745)	0.4534886728220183
  (2, 7324)	0.4760790072128801
  (2, 7168)	0.5655453624599882
  (2, 4618)	0.4798905751162268
  (2, 6650)	0.4724534451200627
  (3, 4599)	0.44821132641606853
  (3, 2099)	0.34395340550548065
  (3, 6689)	0.36608254030361476
  (3, 4078)	0.3578260568100486
  (3, 2111)	0.6471064429181299
  (4, 4078)	0.1668525060013185
  (4, 6469)	0.21602213244654153
  (4, 5836)	0.24512636965486242
  (4, 4171)	0.1875920942459238
  (4, 4589)	0.17768136887787184
  (4, 2899)	0.21063556316852824
  (4, 2670)	0.28648008254733837
  (4, 7315)	0.312571370431853
  (4, 2262)	0.312571370431853
  :	:
  (4454, 997)	0.24283174667869464
  (4454, 5719)	0.17235824583793624
  (4454, 4008)	0.13642682908116005
  (4454, 3963)	0.2795441176182507
  (4454, 2987)	0.21594123389473371
  (4454, 6608)	0.18630215948232687
  (4454, 3165)	0

Logistic regression

In [15]:
# Training a Logistic Regression model
# Logistic regression is a classification algorithm suitable for binary classification tasks
model = LogisticRegression()

# Fit the model on the training data
model.fit(X_train_features, Y_train)

LogisticRegression()

Evaluating the trained model

In [16]:
# Evaluate the model's accuracy on training and test data
# The accuracy score measures how many predictions the model got right
prediction_on_training_data = model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)

# Display the accuracy on the training data
print('Accuracy on training data : ', accuracy_on_training_data)

# Evaluate the accuracy on the test data
prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)

# Display the accuracy on the test data
print('Accuracy on test data : ', accuracy_on_test_data)

Accuracy on training data :  0.97083239847431
Accuracy on test data :  0.9533632286995516


Predictive system

In [17]:
# Sample email input for testing the prediction system
input_mail = ["Hey Sunny I wanted to know if you'll be available this Sunday."]

# Convert the input email to a feature vector using the same TF-IDF transformation as before
input_mail_feature = feature_extraction.transform(input_mail)

# Use the trained model to predict whether the email is spam or not
prediction = model.predict(input_mail_feature)

# Display the prediction result (0: Spam, 1: Ham)
if prediction[0] == 0:
    print('Spam mail')
else:
    print('Ham mail')

Ham mail
